<a href="https://colab.research.google.com/github/AustenLamacraft/learning-wavelets/blob/master/wavelet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-beta1
!pip install -q tensorflow-probability
import tensorflow as tf
import tensorflow_probability as tfp

In [0]:
tf.keras.backend.clear_session()  # For easy reset of notebook state.

In [0]:
class Lifting(Bijector):

  def __init__(self, validate_args=False, name="lifting", n_lifting_coeffs=2):
    super(Exp, self).__init__(
        validate_args=validate_args,
        forward_min_event_ndims=1,
        name=name)
    self.n_lifting_coeffs = n_lifting_coeffs

  def _forward(self, x):
    # TODO

  def _inverse(self, y):
    # TODO

  def _inverse_log_det_jacobian(self, y):
    return 0

  def _forward_log_det_jacobian(self, x):
    return 0

tf.Tensor(
[[ 0.01095894 -0.02076891  0.03318564  0.07053441]
 [ 0.01095894 -0.02076891  0.03318564  0.07053441]], shape=(2, 4), dtype=float32)
